In [1]:
using LinearAlgebra
using Distributions
using FileIO
import JLD2
using DataFrames

In [2]:
using Revise
using Newtrinos
using Newtrinos.osc

In [18]:
include("kathrinADDuDD.jl")

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] (cache misses: wrong dep version loaded (4), incompatible header (2))
ERROR: LoadError: InitError: could not load library "/home/sofialon/.julia/artifacts/a7c26f0f974654c19a165fb76317f44001aa0a29/lib/libwayland-client.so"
libffi.so.6: cannot open shared object file: No such file or directory
Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl ./libdl.jl:120
  [2] dlopen(s::String, flags::UInt32)
    @ Base.Libc.Libdl ./libdl.jl:119
  [3] macro expansion
    @ ~/.julia/packages/JLLWrappers/GfYNv/src/products/library_generators.jl:63 [inlined]
  [4] __init__()
    @ Wayland_jll ~/.julia/packages/Wayland_jll/YZa7L/src/wrappers/x86_64-linux-gnu.jl:16
  [5] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1378
  [6] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1366
  [7] _include_from_serialized(pkg::Base.PkgId, pa

LoadError: LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to "/home/sofialon/.julia/compiled/v1.11/Plots/jl_UAnh3U".
in expression starting at /home/sofialon/Newtrinos.jl/src/experiments/katrin/kathrinADDuDD.jl:2